## Downloading video

In [1]:
! pip install pytube

In [2]:
from pytube import YouTube

In [3]:
%%time
vid = YouTube('https://www.youtube.com/watch?v=wGIN9lSVgl4&ab_channel=2kliksphilip')

CPU times: user 268 µs, sys: 30 µs, total: 298 µs
Wall time: 304 µs


In [4]:
%%time
vid.streams

CPU times: user 5.35 s, sys: 28.9 ms, total: 5.38 s
Wall time: 6.82 s


[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="8fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="315" mime_type="video/webm" res="2160p" fps="60fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="401" mime_type="video/mp4" res="2160p" fps="60fps" vcodec="av01.0.13M.08" progressive="False" type="video">, <Stream: itag="308" mime_type="video/webm" res="1440p" fps="60fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="400" mime_type="video/mp4" res="1440p" fps="60fps" vcodec="av01.0.12M.08" progressive="False" type="video">, <Stream: itag="299" mime_type="video/mp4" res="1080p" fps="60fps" vcodec="avc1.64002a" progressive="False" ty

In [5]:
%%time
vid_streams_filtered = vid.streams.filter(adaptive=True, only_video=True, file_extension="mp4")
print(*vid_streams_filtered, sep="\n")

<Stream: itag="401" mime_type="video/mp4" res="2160p" fps="60fps" vcodec="av01.0.13M.08" progressive="False" type="video">
<Stream: itag="400" mime_type="video/mp4" res="1440p" fps="60fps" vcodec="av01.0.12M.08" progressive="False" type="video">
<Stream: itag="299" mime_type="video/mp4" res="1080p" fps="60fps" vcodec="avc1.64002a" progressive="False" type="video">
<Stream: itag="399" mime_type="video/mp4" res="1080p" fps="60fps" vcodec="av01.0.09M.08" progressive="False" type="video">
<Stream: itag="298" mime_type="video/mp4" res="720p" fps="60fps" vcodec="avc1.4d4020" progressive="False" type="video">
<Stream: itag="398" mime_type="video/mp4" res="720p" fps="60fps" vcodec="av01.0.08M.08" progressive="False" type="video">
<Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="397" mime_type="video/mp4" res="480p" fps="30fps" vcodec="av01.0.04M.08" progressive="False" type="video">
<Stream: itag="134" mime_t

In [6]:
%%time
streams_target_qual = [stream for stream in vid_streams_filtered if 'res="1080p"' in str(stream)]
streams_target_qual 

CPU times: user 72 µs, sys: 1 µs, total: 73 µs
Wall time: 75.1 µs


[<Stream: itag="299" mime_type="video/mp4" res="1080p" fps="60fps" vcodec="avc1.64002a" progressive="False" type="video">,
 <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="60fps" vcodec="av01.0.09M.08" progressive="False" type="video">]

In [7]:
%%time

print(vid_streams_filtered[0].filesize)
print(vid_streams_filtered[1].filesize)

# Afinar el proceso de seleccion en base al tamaño de archivo mas pequeño desde la calidad deseada
# (1080 en este caso) o establecer un limite inferior de calidad y elegir el mas pequeño desde ahi
# escribir el download para que baje el indice de la lista que tenga el menor tamaño


821418395
391578945
CPU times: user 65 µs, sys: 1 µs, total: 66 µs
Wall time: 68.4 µs


In [21]:
%%time
vid_streams_filtered[2].download(filename="aoe2test.mp4")
# usar parametros filename y filename_prefix para etiquetar correctamente los archivos
# output path para mandarlo a una carpeta
# automatizar por playlists para feedear playlist manualmente
# filter through codec, try some codecs to see which ones work

CPU times: user 2.44 s, sys: 643 ms, total: 3.08 s
Wall time: 1min 4s


'/home/julian/code/mauro-vb/game-shazam/pytube_test/aoe2test.mp4'

## Screenshots

In [10]:
!pip install opencv-python

In [22]:
from datetime import timedelta
import cv2
import numpy as np
import os

In [23]:
SAVING_FRAMES_PER_SECOND = 0.25

In [24]:
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

In [25]:
def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [26]:
capture = cv2.VideoCapture('aoe2test.mp4')
capture.release()
cv2.destroyAllWindows()
type(capture)

cv2.VideoCapture

In [27]:
def main(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-frames"
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file    
    capture = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = capture.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(capture, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = capture.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1
    capture.release()
    cv2.destroyAllWindows()

In [ ]:
# PARA CORRERLO DESDE TERMINAL

# if __name__ == "__main__":
#     import sys
#     video_file = sys.argv[1]
#     main(video_file)

In [28]:
main("aoe2test.mp4")